In [2]:
import os
print("Current working directory:", os.getcwd())


Current working directory: c:\Users\rishi\OneDrive\Desktop\New folder\Gen AI\lyrics-generator\backend


converting txt to json

In [3]:

import os
import json

def text_to_json(folder_path):
    data = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            with open(os.path.join(folder_path, file_name), "r") as file:
                data[file_name] = file.read()
    return data

folder_path = "./lyrics/"
json_data = text_to_json(folder_path)

output_path = os.path.join(folder_path, "output.json")
with open(output_path, "w") as json_file:
    json.dump(json_data, json_file, indent=4)

print(f"Conversion completed. JSON file saved as '{output_path}'.")




Conversion completed. JSON file saved as './lyrics/output.json'.


removing all the newlines i.e. "\n"


In [4]:
import json

def remove_newlines_from_json(json_file_path):
    # Read the JSON file
    with open(json_file_path, "r") as file:
        json_data = json.load(file)
    
    # Remove newlines from the JSON data
    json_data_no_newlines = {key: value.replace("\n", "") if isinstance(value, str) else value for key, value in json_data.items()}
    
    # Write the modified JSON data back to the file
    with open(json_file_path, "w") as file:
        json.dump(json_data_no_newlines, file, indent=4)

# Example usage:
json_file_path = "./lyrics/output.json"  # Replace this with the path to your JSON file
remove_newlines_from_json(json_file_path)
print("Newlines removed from the JSON file.")


Newlines removed from the JSON file.


tokenizing the json 

In [5]:
import json
from nltk.tokenize import word_tokenize

# Load text data from JSON file
def load_data_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Tokenize text into words
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

# Path to the JSON file containing text data
json_file_path = './lyrics/output.json'

# Load data from JSON file
data = load_data_from_json(json_file_path)

# Tokenize each text entry in the JSON file
tokenized_data = {}
for key, value in data.items():
    tokenized_data[key] = tokenize_text(value)

# Print tokenized data
for key, tokens in tokenized_data.items():
    print(f"{key}: {tokens}")


Dil Ibadat.txt: ['Dil', 'ibadat', 'kar', 'raha', 'hai', 'Dhadkanein', 'meri', 'sunTujh', 'ko', 'main', 'kar', 'lu', 'haasil', 'lagi', 'hai', 'yehi', 'dhunZindagi', 'ki', 'shaak', 'se', 'loon', 'kuch', 'haseen', 'pal', 'main', 'chunTujh', 'ko', 'main', 'kar', 'lu', 'haasil', 'lagi', 'hai', 'yehi', 'dhunDil', 'ibadat', 'kar', 'raha', 'hai', 'Dhadkanein', 'meri', 'sunTujh', 'ko', 'main', 'kar', 'lu', 'haasil', 'lagi', 'hai', 'yehi', 'dhunZindagi', 'ki', 'shaak', 'se', 'lu', 'kuch', 'haseen', 'pal', 'main', 'chunTujh', 'ko', 'main', 'kar', 'lu', 'haasil', 'lagi', 'hai', 'yehi', 'dhunJo', 'bhi', 'jitne', 'pal', 'jiyun', 'Unhe', 'tere', 'sang', 'jiyunJo', 'bhi', 'kal', 'ho', 'ab', 'mera', 'Usse', 'tere', 'sang', 'jiyunJo', 'bhi', 'saansein', 'main', 'bharun', 'Unhe', 'tere', 'sang', 'bharunChahe', 'jo', 'ho', 'raasta', 'Usse', 'tere', 'sang', 'chalunDil', 'ibadat', 'kar', 'raha', 'hai', 'Dhadkanein', 'meri', 'sunTujh', 'ko', 'main', 'kar', 'lu', 'haasil', 'lagi', 'hai', 'yehi', 'dhunMujhko',

training LSTM Networks with Regularization

In [27]:
import json
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding

# Load text data from JSON file
def load_data_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Preprocess text data
def preprocess_data(data):
    texts = list(data.values())
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    total_words = len(tokenizer.word_index) + 1

    # Convert text to sequences of tokens
    sequences = tokenizer.texts_to_sequences(texts)
    max_sequence_len = max([len(seq) for seq in sequences])

    # Pad sequences to ensure uniform length
    sequences = np.array(pad_sequences(sequences, maxlen=max_sequence_len, padding='pre'))

    return sequences, total_words, max_sequence_len, tokenizer

# Build LSTM model with regularization
def build_model(total_words, max_sequence_len):
    model = Sequential()
    model.add(Embedding(total_words, 100, input_length=max_sequence_len - 1))
    model.add(LSTM(150, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(Dense(total_words / 2, activation='relu'))
    model.add(Dense(total_words, activation='softmax'))
    return model

# Load data from JSON file
json_file_path = './lyrics/output.json'
data = load_data_from_json(json_file_path)

# Preprocess data
sequences, total_words, max_sequence_len, tokenizer = preprocess_data(data)

# Prepare input and output sequences
input_sequences, labels = sequences[:, :-1], sequences[:, -1]

# Build LSTM model
model = build_model(total_words, max_sequence_len)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(input_sequences, labels, epochs=100, batch_size=128, verbose=1)

# Save the trained model
model.save('lyrics_generator_model.h5')

# Save the tokenizer
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


ModuleNotFoundError: No module named 'tensorflow'